In [3]:
-- setup some useful stuff
require 'nn'

-- setting the random generator seed
torch.manualSeed(42)

-- help function to print in green
function cprint(str) print(sys.COLORS.green..str..'\27[0m') end

-- tests
precision = 1e-5
tester = torch.Tester()
function runTest(test)
    tester:add(test)
    tester:run()
    tester = torch.Tester()
end

##Lookup Table

In [37]:
lookupTable = function(vocabSize, embeddingSize)
    -- module to build
    local this = {}
    
    -- standard deviation for initialization
    local stdv =  1./math.sqrt(embeddingSize)
    
    -- weight matrix
    this.weight = torch.Tensor(embeddingSize, vocabSize):uniform(-stdv, stdv)
    
    -- forward operation
    this.forward = function(input)
        -- quirk to make it work with vectors and matrices
        input = (input:dim() == 1) and input:reshape(input:size(1), 1) or input
        -- how many words? how many samples?
        local nWords = input:size(1)
        local nSamples = input:size(2)
        print(nSamples)
        -- view matrix as vector to be used as index
        local vectorView = torch.view(input:long(), -1)
        print(vectorView)
        local output = this.weight:index(2, vectorView)
        print(output)
        return torch.view(output, nSamples, embeddingSize, nWords)
    end
    
    return this
end

-- test
runTest(function()
    print()
    -- 5 words with embeddings of size 3
    local dictionary = lookupTable(5,3)
    dictionary.weight = torch.range(1,15):reshape(5,3):t()
    cprint('LookupTable weight is now:')
    print(dictionary.weight)
    cprint('LookupTable has no bias!')
    tester:assert(dictionary.bias == nil)

    local inputVector = torch.Tensor{1,3,5}
    cprint('Testing input vector is:')
    print(inputVector)
    local expected = torch.Tensor{{1,2,3},{7,8,9},{13,14,15}}:t():reshape(1, 3, 3)
    cprint('Expected output is:')
    print(expected)
    local output = dictionary.forward(inputVector)
    cprint('Actual output is:')
    print(output)
    tester:assertTensorEq(expected, output, precision)

    local inputMatrix = torch.Tensor{{1,3},{2,4}}:t()
    cprint('Testing input matrix is:')
    print(inputMatrix)
    local expected = torch.Tensor{{1,2,3},{7,8,9},{4,5,6},{10,11,12}}:reshape(2, 2, 3):transpose(2, 3)
    cprint('Expected output is:')
    print(expected)
    local output = dictionary.forward(inputMatrix)
    cprint('Actual output is:')
    print(output)
    tester:assertTensorEq(expected, output, precision)
end)

Running 1 tests	
|  ==> unknown
LookupTable weight is now:	
  1   4   7  10  13
  2   5   8  11  14
  3   6   9  12  15
[torch.DoubleTensor of dimension 3x5]

LookupTable has no bias!	
Testing input matrix is:	
 1  2
 3  4
[torch.DoubleTensor of dimension 2x2]

Expected output is:	


(1,.,.) = 
   1   7
   2   8
   3   9

(2,.,.) = 
   4  10
   5  11
   6  12
[torch.DoubleTensor of dimension 2x3x2]

2	
 1
 2
 3
 4
[torch.LongTensor of dimension 4]

  1   2   3
  4   5   6
  7   8   9
 10  11  12
[torch.DoubleTensor of dimension 4x3]



*  ==> Done 

Completed 1 asserts in 1 tests with 1 errors	

--------------------------------------------------------------------------------	
unknown
 Function call failed 
/home/fiskio/torch/install/share/lua/5.1/torch/Tensor.lua:450: expecting a contiguous tensor
stack traceback:
	[C]: in function 'assert'
	/home/fiskio/torch/install/share/lua/5.1/torch/Tensor.lua:450: in function 'forward'
	[string "lookupTable = function(vocabSize, embeddingSi..."]:58: in function <[string "lookupTable = function(vocabSize, embeddingSi..."]:31>
	[C]: in function 'xpcall'
	/home/fiskio/torch/install/share/lua/5.1/torch/Tester.lua:112: in function 'pcall'
	/home/fiskio/torch/install/share/lua/5.1/torch/Tester.lua:169: in function 'run'
	[string "-- setup some useful stuff..."]:15: in function 'runTest'
	[string "lookupTable = function(vocabSize, embeddingSi..."]:31: in main chunk
	[C]: in function 'xpcall'
	/home/fiskio/torch/install/share/lua/5.1/itorch/main.lua:174: in function </home/fiskio/torch

##Linear Module

In [7]:
linearModule = function(inputSize, outputSize)
    
    -- module to build
    local this = {}
    
    -- standard deviation for initialization
    local stdv =  1./math.sqrt(outputSize)
    
    -- weight matrix
    this.weight = torch.Tensor(outputSize, inputSize):uniform(-stdv, stdv)
    
    -- bias vector
    this.bias = torch.Tensor(outputSize, 1):uniform(-stdv, stdv)
    
    -- forward operation
    this.forward = function(input)
        -- quirk to make it work with vectors and matrices
        input = (input:dim() == 1) and input:reshape(input:size(1), 1) or input
        -- multiply the input and weight matrix
        local output = this.weight * input
        -- add the expanded bias vector and return
        return output + this.bias:expand(output:size())
    end

    return this
end

-- test
runTest(function()
    print()
    local layer = linearModule(2,3)
    layer.weight:fill(2)
    layer.bias:fill(1)
    cprint('LinearModule weight is now:')
    print(layer.weight)
    cprint('LinearModule bias is now:')
    print(layer.bias)

    local inputVector = torch.Tensor(2):fill(-1)
    cprint('Testing input vector is:')
    print(inputVector)
    local expected = torch.Tensor(3,1):fill(-3)
    cprint('Expected output is:')
    print(expected)
    local output = layer.forward(inputVector)
    cprint('Actual output is:')
    print(output)
    tester:assertTensorEq(expected, output, precision)

    local inputMatrix = torch.Tensor(2, 3):fill(2)
    cprint('Testing input matrix is:')
    print(inputMatrix)
    local expected = torch.Tensor(3,3):fill(9)
    cprint('Expected output is:')
    print(expected)
    local output = layer.forward(inputMatrix)
    cprint('Actual output is:')
    print(output)
    tester:assertTensorEq(expected, output, precision)
end)

Running 1 tests	
|  ==> unknown
LinearModule weight is now:	
 2  2
 2  2
 2  2
[torch.DoubleTensor of dimension 3x2]

LinearModule bias is now:	
 1
 1
 1
[torch.DoubleTensor of dimension 3x1]

Testing input vector is:	
-1
-1
[torch.DoubleTensor of dimension 2]

Expected output is:	
-3
-3
-3
[torch.DoubleTensor of dimension 3x1]

Actual output is:	
-3
-3
-3
[torch.DoubleTensor of dimension 3x1]

Testing input matrix is:	
 2  2  2
 2  2  2
[torch.DoubleTensor of dimension 2x3]

Expected output is:	
 9  9  9
 9  9  9
 9  9  9
[torch.DoubleTensor of dimension 3x3]

Actual output is:	
 9  9  9
 9  9  9
 9  9  9
[torch.DoubleTensor of dimension 3x3]



_  ==> Done 

Completed 2 asserts in 1 tests with 0 errors	

--------------------------------------------------------------------------------	


##Sigmoid
\begin{equation*}
    Sigmoid(x_i) = \frac{1}{1 + e^{-x_i}}
\end{equation*}

In [12]:
sigmoid = function(input)
    return torch.exp(input):pow(-1):add(1):pow(-1)
end

-- test
runTest(function()
    print()
    local inputVector = torch.range(1,3)
    cprint('Testing input vector is:')
    print(inputVector)
    local expected = nn.Sigmoid():forward(inputVector)
    cprint('Expected output is:')
    print(expected)
    local output = sigmoid(inputVector)
    cprint('Actual output is:')
    print(output)
    tester:assertTensorEq(expected, output, precision)
        
    local inputMatrix = torch.range(1,9):reshape(3,3)
    cprint('Testing input matrix is:')
    print(inputMatrix)
    local expected = nn.Sigmoid():forward(inputMatrix)
    cprint('Expected output is:')
    print(expected)
    local output = sigmoid(inputMatrix)
    cprint('Actual output is:')
    print(output)
    tester:assertTensorEq(expected, output, precision)
end)

Running 1 tests	
|  ==> unknown
Testing input vector is:	
 1
 2
 3
[torch.DoubleTensor of dimension 3]

Expected output is:	
 0.7311
 0.8808
 0.9526
[torch.DoubleTensor of dimension 3]

Actual output is:	
 0.7311
 0.8808
 0.9526
[torch.DoubleTensor of dimension 3]

Testing input matrix is:	
 1  2  3
 4  5  6
 7  8  9
[torch.DoubleTensor of dimension 3x3]

Expected output is:	
 0.7311  0.8808  0.9526
 0.9820  0.9933  0.9975
 0.9991  0.9997  0.9999
[torch.DoubleTensor of dimension 3x3]

Actual output is:	
 0.7311  0.8808  0.9526
 0.9820  0.9933  0.9975
 0.9991  0.9997  0.9999
[torch.DoubleTensor of dimension 3x3]



_  ==> Done 

Completed 2 asserts in 1 tests with 0 errors	

--------------------------------------------------------------------------------	


## LogSoftMax
\begin{equation*}
   LogSoftMax(x_i) = -\ln \Bigl(\frac{1}{e^{x_i}} \sum_j e^{x_j}\Bigr)
\end{equation*}

In [47]:
logSoftMax = function(input) 
    
    -- quirk to make it work with vectors and matrices
    input = (input:dim() == 1) and input:reshape(input:size(1), 1) or input
    
    -- calculate sum of e^x_i and expand it to the right size
    local sumOfExp = torch.exp(input):sum(2):expand(input:size())
    
    -- calculate the rest of the formula and return
    return torch.exp(input):pow(-1):cmul(sumOfExp):log():mul(-1)
end

-- test
runTest(function()
    print()
    input = torch.range(1,9):reshape(3,3)
    cprint('Input matrix is:')
    print(input)
    local lms = nn.LogSoftMax()
    local expected = lms:forward(input)
    cprint('Output from nn.LogSoftMax is:')
    print(expected)
    local output = logSoftMax(input)
    cprint('Output from logSoftMax is:')
    print(output)
    tester:assertTensorEq(expected, output, 1e-5)
end)

Running 1 tests	
|  ==> unknown
Input matrix is:	
 1  2  3
 4  5  6
 7  8  9
[torch.DoubleTensor of dimension 3x3]



Output from nn.LogSoftMax is:	
-2.4076 -1.4076 -0.4076
-2.4076 -1.4076 -0.4076
-2.4076 -1.4076 -0.4076
[torch.DoubleTensor of dimension 3x3]

Output from logSoftMax is:	
-2.4076 -1.4076 -0.4076
-2.4076 -1.4076 -0.4076
-2.4076 -1.4076 -0.4076
[torch.DoubleTensor of dimension 3x3]

_  ==> Done 

Completed 1 asserts in 1 tests with 0 errors	

--------------------------------------------------------------------------------	


##Negative Log-Likelihood

\begin{equation*}
    C = -\ln a^L_y
\end{equation*}

In [ ]:
negativeLogLikelihood = function(input, class)
    return input:log():mul(-1)[class]
end

-- test

In [1]:
-- network parameters
local embeddingSize = 2
local contextLength = 3
local vocabSize = 5
local hiddenSize = 6
local stdv = 1

-- IndexToEmbedding
local lookupTable = linearModule(vocabSize, embeddingSize)
cprint('LookupTable of '..vocabSize..' words, each is a vector of size '..embeddingSize)
print(lookupTable.weight)
print(lookupTable.bias)

-- ContextToHidden
local contextToHidden = linearModule(contextLength * embeddingSize, hiddenSize)
cprint('Context-To-Hidden matrix is:')
print(contextToHidden.weight)
print(contextToHidden.bias)

local hiddenToEmbedding = torch.FloatTensor(embeddingSize, hiddenSize):uniform(-stdv, stdv)

cprint('Hidden-To-Embedding matrix is:')
print(hiddenToEmbedding)

local embeddingToVocabulary = torch.FloatTensor(vocabSize, embeddingSize):uniform(-stdv, stdv)

cprint('Embedding-To-Vocabulary matrix is:')
print(embeddingToVocabulary)

function softMax(matrix)
   -- -log(sum(exp(matrix)) * 1/exp(matrix))
   return torch.mul(torch.exp(matrix):pow(-1), torch.sum(torch.exp(matrix), 1)[1]):log():mul(-1)
end

-- Forward
--[[
local oneHot = torch.FloatTensor():eye(vocabSize)
cprint('1-Hot representation of second word is:')
print(oneHot[2])

local secondWord = torch.mv(lookupTable, oneHot[2])
cprint('Vector representation of second word is:')
print(secondWord)
--]]
local oneHotIndices = torch.LongTensor{1,3,5}
cprint('Context will be built of words at indices...')
print(oneHotIndices)

local contextMatrix = lookupTable:index(2, oneHotIndices)
cprint('...which corresponds to the following matrix:')
print(oneHotContext)
--[[
local contextMatrix = torch.mm(lookupTable, oneHotContext)
cprint('The corresponding matrix of embeddings are:')
print(contextMatrix)
--]]
local contextVector = torch.reshape(contextMatrix, contextMatrix:nElement())
cprint('...which reshaped as a vector is:')
print(contextVector)

local output = contextToHidden * contextVector
print(output)

output = torch.mv(hiddenToEmbedding, output)
print(output)

output = torch.mv(embeddingToVocabulary, output)
print(output)

output = softMax(output)
print(output)


[string "-- network parameters..."]:9: attempt to call global 'linearModule' (a nil value)
stack traceback:
	[string "-- network parameters..."]:9: in main chunk
	[C]: in function 'xpcall'
	/home/fiskio/torch/install/share/lua/5.1/itorch/main.lua:174: in function </home/fiskio/torch/install/share/lua/5.1/itorch/main.lua:140>
	/home/fiskio/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/home/fiskio/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/home/fiskio/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/home/fiskio/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/home/fiskio/torch/install/share/lua/5.1/itorch/main.lua:341: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x00406170: 